- references
    - https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model

In [1]:
from IPython.display import Image

In [3]:
Image(url='https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F0605ace1-cc21-400a-941f-154a1faa103c_1600x1427.png', width=400)